In [46]:
import numpy as numpy
import pandas as pd
import matplotlib as mpl
import sklearn.linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [47]:
# Features we were given
hog_pca = pd.read_csv("train/Features/hog_pca.csv")
color_hist = pd.read_csv("train/Features/color_histogram.csv")
additional_features = pd.read_csv("train/Features/additional_features.csv")
train_metadata = pd.read_csv("train/train_metadata.csv")

In [48]:
# Merge color_hist into hog_pca
hog_pca = color_hist

hog_pca = hog_pca.merge(color_hist, on='image_path')

# Merge additional_features into the result
hog_pca = hog_pca.merge(additional_features, on='image_path')

In [49]:
from sklearn.preprocessing import MinMaxScaler

# Create a scaler
scaler = MinMaxScaler()

# Select numeric columns (excluding 'image_path')
features = hog_pca.columns.drop('image_path')

# Fit and transform the numeric columns
hog_pca[features] = scaler.fit_transform(hog_pca[features])

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
hog_pca[features] = scaler.fit_transform(hog_pca[features])

In [50]:
hog_pca

,image_path,ch_0_x,ch_1_x,ch_2_x,ch_3_x,ch_4_x,ch_5_x,ch_6_x,ch_7_x,ch_8_x,...,ch_90_y,ch_91_y,ch_92_y,ch_93_y,ch_94_y,ch_95_y,edge_density,mean_b,mean_g,mean_r
0,img_000001.jpg,-0.092757,-0.409911,0.118950,0.364295,1.181858,0.236360,2.014151,1.129331,-0.266137,...,-0.547539,-0.536360,-0.523940,-0.549625,-0.560424,-0.586172,-0.572368,-0.391041,-0.379058,-0.481654
1,img_000002.jpg,-0.092757,-0.399719,1.414734,1.474258,-0.113854,0.123534,-0.458289,-0.809185,-0.845642,...,-0.547539,-0.536360,-0.523940,-0.549625,-0.560424,-0.586172,-0.791843,-0.710490,-0.574687,-0.507331
2,img_000003.jpg,0.037730,0.235497,0.508236,-0.659509,-0.816047,-0.078118,0.989447,0.429520,0.692837,...,0.648475,0.882914,1.037071,1.536037,2.070787,1.636411,0.484485,0.360030,0.651803,0.781085
3,img_000004.jpg,-0.092757,0.022046,2.379893,-0.172600,-0.393301,-0.704064,-0.846104,-0.752132,-0.700671,...,-0.547539,-0.536360,-0.523940,-0.549625,-0.560424,-0.586172,-0.906645,-1.096067,-1.130844,-1.144259
4,img_000005.jpg,-0.092757,-0.409911,-0.776775,-1.016575,-1.112842,-1.054104,-0.921046,-0.891091,-0.260610,...,-0.050437,-0.068409,-0.310168,-0.421904,-0.447314,-0.536466,0.210986,0.734484,0.374533,-0.155781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5483,img_005484.jpg,-0.092757,-0.386919,-0.563861,-0.080058,0.978313,1.673404,0.835894,-0.456331,1.938299,...,-0.129398,0.381031,0.817145,1.440402,1.990390,1.945135,0.123197,0.330848,0.581534,1.021180
5484,img_005485.jpg,-0.092757,-0.409911,-0.776775,-1.016575,-1.112842,-1.133329,-1.097824,-1.060094,-1.010867,...,-0.380516,-0.321574,-0.337876,-0.352767,-0.396341,0.081033,-0.795219,1.890378,1.628126,1.484391
5485,img_005486.jpg,0.245169,-0.204713,0.091116,-0.033872,-0.468913,-0.579922,-0.622240,-0.568343,-0.533363,...,0.020740,-0.066346,0.381526,1.306685,2.549009,2.562609,2.155866,1.311300,1.657168,1.827534
5486,img_005487.jpg,-0.092757,1.858592,0.613975,0.432723,0.845617,-0.529558,-0.786630,-0.841719,-0.841027,...,3.935775,-0.221460,-0.523940,-0.549625,-0.560424,-0.586172,-0.731065,0.159418,0.092547,-0.073219


In [51]:
train = pd.merge(hog_pca, train_metadata, on = 'image_path')

In [52]:
train

,image_path,ch_0_x,ch_1_x,ch_2_x,ch_3_x,ch_4_x,ch_5_x,ch_6_x,ch_7_x,ch_8_x,...,ch_92_y,ch_93_y,ch_94_y,ch_95_y,edge_density,mean_b,mean_g,mean_r,id,ClassId
0,img_000001.jpg,-0.092757,-0.409911,0.118950,0.364295,1.181858,0.236360,2.014151,1.129331,-0.266137,...,-0.523940,-0.549625,-0.560424,-0.586172,-0.572368,-0.391041,-0.379058,-0.481654,7218,32
1,img_000002.jpg,-0.092757,-0.399719,1.414734,1.474258,-0.113854,0.123534,-0.458289,-0.809185,-0.845642,...,-0.523940,-0.549625,-0.560424,-0.586172,-0.791843,-0.710490,-0.574687,-0.507331,6333,13
2,img_000003.jpg,0.037730,0.235497,0.508236,-0.659509,-0.816047,-0.078118,0.989447,0.429520,0.692837,...,1.037071,1.536037,2.070787,1.636411,0.484485,0.360030,0.651803,0.781085,6867,7
3,img_000004.jpg,-0.092757,0.022046,2.379893,-0.172600,-0.393301,-0.704064,-0.846104,-0.752132,-0.700671,...,-0.523940,-0.549625,-0.560424,-0.586172,-0.906645,-1.096067,-1.130844,-1.144259,7730,4
4,img_000005.jpg,-0.092757,-0.409911,-0.776775,-1.016575,-1.112842,-1.054104,-0.921046,-0.891091,-0.260610,...,-0.310168,-0.421904,-0.447314,-0.536466,0.210986,0.734484,0.374533,-0.155781,6338,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5483,img_005484.jpg,-0.092757,-0.386919,-0.563861,-0.080058,0.978313,1.673404,0.835894,-0.456331,1.938299,...,0.817145,1.440402,1.990390,1.945135,0.123197,0.330848,0.581534,1.021180,456,17
5484,img_005485.jpg,-0.092757,-0.409911,-0.776775,-1.016575,-1.112842,-1.133329,-1.097824,-1.060094,-1.010867,...,-0.337876,-0.352767,-0.396341,0.081033,-0.795219,1.890378,1.628126,1.484391,4084,4
5485,img_005486.jpg,0.245169,-0.204713,0.091116,-0.033872,-0.468913,-0.579922,-0.622240,-0.568343,-0.533363,...,0.381526,1.306685,2.549009,2.562609,2.155866,1.311300,1.657168,1.827534,5117,2
5486,img_005487.jpg,-0.092757,1.858592,0.613975,0.432723,0.845617,-0.529558,-0.786630,-0.841719,-0.841027,...,-0.523940,-0.549625,-0.560424,-0.586172,-0.731065,0.159418,0.092547,-0.073219,3334,3


In [53]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Split features and target
X = train.drop(columns=['image_path', 'ClassId'])  # adjust 'label' if your column has a different name
y = train['ClassId']

# Step 2: Train-test split (optional but recommended)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Create and fit KNN model
knn = KNeighborsClassifier(n_neighbors=10)  # you can tune n_neighbors
knn.fit(X_train, y_train)

# Step 4: Predict and evaluate
y_pred = knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.05555555555555555
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.10      0.28      0.14        58
           2       0.04      0.10      0.06        61
           3       0.07      0.16      0.10        37
           4       0.07      0.13      0.09        54
           5       0.04      0.06      0.05        52
           6       0.00      0.00      0.00        11
           7       0.04      0.05      0.04        42
           8       0.03      0.03      0.03        29
           9       0.00      0.00      0.00        41
          10       0.10      0.13      0.12        54
          11       0.03      0.03      0.03        37
          12       0.06      0.07      0.07        55
          13       0.04      0.03      0.04        60
          14       0.00      0.00      0.00        24
          15       0.00      0.00      0.00        18
          16       0.00    

c:\Users\endah\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\endah\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\endah\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo